<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/eng_sin_mt5_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda update -n base -c conda-forge conda
!conda --version
!conda create -n st python pandas tqdm sacrebleu
!conda activate st
!conda install pytorch>=1.6 cudatoolkit=10.2 -n base -c conda-forge conda pytorch

✨🍰✨ Everything looks OK!
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ 

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r '/content/drive/MyDrive/data' '/content'

Mounted at /content/drive


In [3]:
import os
import pandas as pd

In [4]:
def prepare_translation_datasets(data_path):
    with open(os.path.join(data_path, "train.trg"), "r", encoding="utf-8") as f:
        sinhala_text = f.readlines()
        sinhala_text = [text.strip("\n") for text in sinhala_text]

    with open(os.path.join(data_path, "train.src"), "r") as f:
        english_text = f.readlines()
        english_text = [text.strip("\n") for text in english_text]

    data = []
    for sinhala, english in zip(sinhala_text, english_text):
        data.append(["translate sinhala to english", sinhala, english])
        data.append(["translate english to sinhala", english, sinhala])

    train_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])

    with open(os.path.join(data_path, "test.trg"), "r", encoding="utf-8") as f:
        sinhala_text = f.readlines()
        sinhala_text = [text.strip("\n") for text in sinhala_text]

    with open(os.path.join(data_path, "test.src"), "r") as f:
        english_text = f.readlines()
        english_text = [text.strip("\n") for text in english_text]

    data = []
    for sinhala, english in zip(sinhala_text, english_text):
        data.append(["translate sinhala to english", sinhala, english])
        data.append(["translate english to sinhala", english, sinhala])

    eval_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])

    return train_df, eval_df

In [5]:
train_df, eval_df = prepare_translation_datasets("data/eng-sin")

In [6]:
train_df

,prefix,input_text,target_text
0,translate sinhala to english,මෙය සිදු වන්නේ කවදාද?,When will this happen?
1,translate english to sinhala,When will this happen?,මෙය සිදු වන්නේ කවදාද?
2,translate sinhala to english,දුවන්න !,Run!
3,translate english to sinhala,Run!,දුවන්න !
4,translate sinhala to english,- අනිද්දට,- The day after tomorrow.
...,...,...,...
2255049,translate english to sinhala,"As those words indicate, some Christians will ...",ඒ විරුද්ධවාදිකම් ආගමික නැත්නම් දේශපාලන නයකයන්ග...
2255050,translate sinhala to english,එතකොට ඔයා මොකද කරන්නේ?,Then what are you to do?
2255051,translate english to sinhala,Then what are you to do?,එතකොට ඔයා මොකද කරන්නේ?
2255052,translate sinhala to english,"ඔහු ජාන විද්‍යාඥයෙක්, මානව ජානවිද්‍යාව ගැන විශ...","Only that he's a genecist, an expert on human ..."


In [7]:
train_df[:8000].to_csv("data/train.tsv", sep="\t")
eval_df.to_csv("data/eval.tsv", sep="\t")

In [8]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 431.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 112.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 111.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 1

In [1]:
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [2]:
train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("data/eval.tsv", sep="\t").astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

In [3]:
!pip install torch

import torch

device = torch.cuda.is_available()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 890.2/890.2 MB 199.2 MB/s eta 0:00:01tcmalloc: large alloc 1112711168 bytes == 0x55d12a034000 @  0x7fad7d54d615 0x55d0f1a31857 0x55d0f1a9a14b 0x55d0f1a99feb 0x55d0f1a13d33 0x55d0f19f5e8d 0x55d0f19f4551 0x55d0f1a063fc 0x55d0f19f5e8d 0x55d0f19f4551 0x55d0f1a063fc 0x55d0f19f5e8d 0x55d0f19f4551 0x55d0f1a063fc 0x55d0f19f5e8d 0x55d0f19f4551 0x55d0f1a063fc 0x55d0f19f5e8d 0x55d0f19f4551 0x55d0f1a063fc 0x55d0f19f5e8d 0x55d0f1a06366 0x55d0f19f5e8d 0x55d0f19f4551 0x55d0f1a063fc 0x55d0f19f68f8 0x55d0f19f4551 0x55d0f1a063fc 0x55d0f19f5e8d 0x55d0f19f4551 0x55d0f1a063fc
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849

In [4]:
model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 20
model_args.eval_batch_size = 20
model_args.num_train_epochs = 1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Sinhala-English Translation"

device = torch.cuda.is_available()
model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=device)

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

In [5]:
# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/8000 [00:00<?, ?it/s]

/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 1075, in init
    wi.setup(kwargs)
  File "/usr/local/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 165, in setup
    self._wl = wandb_setup.setup()
  File "/usr/local/lib/python3.8/site-packages/wandb/sdk/wandb_setup.py", line 312, in setup
    ret = _setup(settings=settings)
  File "/usr/local/lib/python3.8/site-packages/wandb/sdk/wandb_setup.py", line 307, in _setup
    wl = _WandbSetup(settings=settings)
  File "/usr/local/lib/python3.8/site-packages/wandb/sdk/wandb_setup.py", line 293, in __init__
    _WandbSetup._instance = _WandbSetup__WandbSetup(settings=settings, pid=pid)
  File "/usr/local/lib/python3.8/site-packages/wandb/sdk/wandb_setup.py", line 106, in __init__
    self._setup()
  File "/us

proc exited with 1


Exception: ignored